In [3]:
import pandas as pd
from functools import reduce
import numpy as np
import impyute as impy
from sklearn.metrics import mean_squared_error,r2_score
import datawig

#### Data Preparation

In [2]:
def readtodf(filename, colname):
    a=filename+'.txt'
    data = pd.read_csv(a, sep=",", header=(0))
    data.columns=['a','Date',colname]
    data=data.drop('a',axis=1)
    data['Date'] = pd.to_datetime(data['Date'])
    data = data.set_index('Date')
    
    return data

In [4]:
target=readtodf('prod_target','Beer')
prod_1=readtodf('prod_1','Car')
prod_2=readtodf('prod_2','Steel')
eng_1=readtodf('eng_1','Gas')
eng_2=readtodf('eng_2','Electricity')

In [5]:
temp = pd.read_csv('temp.txt', sep=",", header=(0))
temp.columns=['num','year','month','Temp']
temp=temp.drop('num',axis=1)
temp['day']=1
temp['Date']=pd.to_datetime(temp[['year', 'month', 'day']])
temp=temp.drop(['year', 'month', 'day'],axis=1)
temp = temp.set_index('Date')

In [6]:
target.shape,prod_1.shape,prod_2.shape, eng_1.shape,eng_2.shape,temp.shape

((435, 1), (369, 1), (435, 1), (435, 1), (435, 1), (581, 1))

In [7]:
 # creat dataframe version of merged data
dfs = [target, prod_1, prod_2, eng_1,eng_2,temp]
df_final = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True,how='left'), dfs)
#df_final.to_csv('data_merged.csv')

# creat numpy version of merged data
np_final=np.array(df_final.values,dtype=np.float)

In [20]:
df_final.shape

(435, 6)

In [19]:
df_final.head()

,Beer,Car,Steel,Gas,Electricity,Temp
Date,,,,,,
1956-01-01,93.2,NaN,196.9,1709,1254,25.1
1956-02-01,96.0,NaN,192.1,1646,1290,25.3
1956-03-01,95.2,NaN,201.8,1794,1379,24.9
1956-04-01,77.1,NaN,186.9,1878,1346,23.9
1956-05-01,70.9,NaN,218.0,2173,1535,19.4


### Imputation

##### 1.Mice

The Multiple Imputation by Chained Equations (MICE) method is widely used in practice, which uses chain equations to create multiple imputations for variables of diferent types.

In [8]:
ip_mice=impy.imputation.cs.mice(np_final)
np.savetxt('./Imputation Results/imputation_mice.csv',ip_mice,delimiter=",")

##### 2.KNN

- This method uses k-nearest neighbor to fInd similar samples and imputed unobserved data by weighted average of similar observations.
- Basic idea: Impute array with a basic mean impute and then use the resulting complete array to construct a KDTree. Use this KDTree to compute nearest neighbours. After finding k nearest neighbours, take the weighted average of them. Basically, find the nearest row in terms of distance

In [10]:
ip_knn=impy.imputation.cs.fast_knn(np_final)
np.savetxt('./Imputation Results/imputation_knn.csv',ip_knn,delimiter=",")

##### 3.DataWig

- "Deep" Learning for Missing Value Imputationin Tables with Non-Numerical Data
- Details on the underlying model can be found in [Biessmann, Salinas et al. 2018](https://dl.acm.org/citation.cfm?id=3272005)

In [128]:
#Initialize a SimpleImputer model
imputer = datawig.SimpleImputer(
    input_columns=['Car','Steel','Gas','Electricity','Temp'], # column(s) containing information about the column we want to impute
    output_column='Beer', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )

#Using LSTMs instead of bag-of-words
# data_encoder_cols = [NumericalEncoder('Car'), NumericalEncoder('Steel'),NumericalEncoder('Gas'),\
#                     NumericalEncoder('Electricity'),NumericalEncoder('Temp')]
# label_encoder_cols = [NumericalEncoder('Beer')]
# data_featurizer_cols = [LSTMFeaturizer('Car'), LSTMFeaturizer('Steel'),LSTMFeaturizer('Gas'),\
#                     LSTMFeaturizer('Electricity'),LSTMFeaturizer('Temp')]

# imputer = Imputer(
#     data_featurizers=data_featurizer_cols,
#     label_encoders=label_encoder_cols,
#     data_encoders=data_encoder_cols,
#     output_path='imputer_model'
# )

In [137]:
#Fit an imputer model on the train data
imputer.fit(train_df=df_final[df_final['Beer'].notnull()], num_epochs=300,learning_rate=1e-3,batch_size=5,weight_decay=1e-4)

2019-04-20 23:21:52,585 [INFO]  Assuming 5 numeric input columns: Car, Steel, Gas, Electricity, Temp
2019-04-20 23:21:52,589 [INFO]  Assuming 0 string input columns: 
2019-04-20 23:21:52,593 [INFO]  No output column name provided for ColumnEncoder using Beer
2019-04-20 23:21:52,596 [INFO]  Assuming numeric output column: Beer
2019-04-20 23:21:52,599 [INFO]  Using [[cpu(0)]] as the context for training
2019-04-20 23:21:52,605 [INFO]  Detected 0 rows with missing labels                         for column Beer
2019-04-20 23:21:52,608 [INFO]  Dropping 0/364 rows
2019-04-20 23:21:52,611 [INFO]  Detected 0 rows with missing labels                         for column Beer
2019-04-20 23:21:52,614 [INFO]  Dropping 0/40 rows
2019-04-20 23:21:52,617 [INFO]  Train: 364, Test: 40
2019-04-20 23:21:52,619 [INFO]  Building Train Iterator with 364 elements
2019-04-20 23:21:52,637 [INFO]  Concatenating numeric columns ['Car', 'Steel', 'Gas', 'Electricity', 'Temp'] into numerical_features-9ZMJLsCfMZ
2019-

In [138]:
#Impute missing values and return original dataframe with predictions
imputed = imputer.predict(df_final)
#imputed.to_csv('./Imputation Results/imputation_Datawig.csv')

2019-04-20 23:21:55,699 [INFO]  Concatenating numeric columns ['Car', 'Steel', 'Gas', 'Electricity', 'Temp'] into numerical_features-9ZMJLsCfMZ
2019-04-20 23:21:55,701 [INFO]  Normalizing with StandardScaler
2019-04-20 23:21:55,706 [INFO]  Data Encoding - Encoded 435 rows of column                         Car,Steel,Gas,Electricity,Temp with <class 'datawig.column_encoders.NumericalEncoder'> into                         <class 'numpy.ndarray'> of shape (435, 5)                         and then into shape (435, 5)
2019-04-20 23:21:55,711 [INFO]  Concatenating numeric columns ['Beer'] into Beer
2019-04-20 23:21:55,714 [INFO]  Normalizing with StandardScaler
2019-04-20 23:21:55,718 [INFO]  Label Encoding - Encoded 435 rows of column                             Beer with <class 'datawig.column_encoders.NumericalEncoder'> into                             <class 'numpy.ndarray'> of shape (435, 1) and                             then into shape (435, 1)
2019-04-20 23:21:55,776 [INFO]  Top-k on

In [139]:
predictions=imputed[imputed['Beer'].notnull()]

In [140]:
#Calculate MSE score
MSE = mean_squared_error(predictions['Beer'].values, predictions['Beer_imputed'].values)

#Calculate r2 score
r2=r2_score(predictions['Beer'].values, predictions['Beer_imputed'].values)

MSE,r2

(198.71291211265773, 0.836286238218155)

In [59]:
imputed_data=imputed.copy()
imputed_data.loc['1972-09-01':'1975-02-01','Beer']=imputed.loc['1972-09-01':'1975-02-01']['Beer_imputed'].values
imputed_data=imputed_data.drop('Beer_imputed',axis=1);

In [60]:
imputed_data.head()

,Beer,Car,Steel,Gas,Electricity,Temp
Date,,,,,,
1956-01-01,93.2,NaN,196.9,1709,1254,25.1
1956-02-01,96.0,NaN,192.1,1646,1290,25.3
1956-03-01,95.2,NaN,201.8,1794,1379,24.9
1956-04-01,77.1,NaN,186.9,1878,1346,23.9
1956-05-01,70.9,NaN,218.0,2173,1535,19.4


In [54]:
#Initialize a SimpleImputer model
imputer = datawig.SimpleImputer(
    input_columns=['Beer','Steel','Gas','Electricity','Temp'], # column(s) containing information about the column we want to impute
    output_column='Car', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )

In [55]:
#Fit an imputer model on the train data
imputer.fit(train_df=imputed_data[imputed_data['Car'].notnull()], num_epochs=300)

2019-04-20 22:44:42,708 [INFO]  Assuming 5 numeric input columns: Beer, Steel, Gas, Electricity, Temp
2019-04-20 22:44:42,710 [INFO]  Assuming 0 string input columns: 
2019-04-20 22:44:42,712 [INFO]  No output column name provided for ColumnEncoder using Car
2019-04-20 22:44:42,713 [INFO]  Assuming numeric output column: Car
2019-04-20 22:44:42,715 [INFO]  Using [[cpu(0)]] as the context for training
2019-04-20 22:44:42,720 [INFO]  Fitting label encoder <class 'datawig.column_encoders.NumericalEncoder'> on 332 rows                             of training data
2019-04-20 22:44:42,728 [INFO]  Detected 0 rows with missing labels                         for column Car
2019-04-20 22:44:42,730 [INFO]  Dropping 0/332 rows
2019-04-20 22:44:42,733 [INFO]  Detected 0 rows with missing labels                         for column Car
2019-04-20 22:44:42,735 [INFO]  Dropping 0/36 rows
2019-04-20 22:44:42,738 [INFO]  Train: 332, Test: 36
2019-04-20 22:44:42,739 [INFO]  Fitting data encoder <class 'dat

In [56]:
#Impute missing values and return original dataframe with predictions
imputed_car = imputer.predict(imputed_data)
#imputed.to_csv('./Imputation Results/imputation_Datawig.csv')

2019-04-20 22:44:56,438 [INFO]  Concatenating numeric columns ['Beer', 'Steel', 'Gas', 'Electricity', 'Temp'] into numerical_features-BuNO6n9JEC
2019-04-20 22:44:56,439 [INFO]  Normalizing with StandardScaler
2019-04-20 22:44:56,443 [INFO]  Data Encoding - Encoded 448 rows of column                         Beer,Steel,Gas,Electricity,Temp with <class 'datawig.column_encoders.NumericalEncoder'> into                         <class 'numpy.ndarray'> of shape (448, 5)                         and then into shape (448, 5)
2019-04-20 22:44:56,448 [INFO]  Concatenating numeric columns ['Car'] into Car
2019-04-20 22:44:56,451 [INFO]  Normalizing with StandardScaler
2019-04-20 22:44:56,454 [INFO]  Label Encoding - Encoded 448 rows of column                             Car with <class 'datawig.column_encoders.NumericalEncoder'> into                             <class 'numpy.ndarray'> of shape (448, 1) and                             then into shape (448, 1)
2019-04-20 22:44:56,472 [INFO]  Top-k onl

In [142]:
imputed_data_final=imputed_car.copy()
imputed_data_final.loc['1956-01-01':'1961-06-01','Car']=\
imputed_car.loc['1956-01-01':'1961-06-01']['Car_imputed'].values
imputed_data_final=imputed_data_final.drop('Car_imputed',axis=1)
imputed_data_final.to_csv('data_merged_final.csv');